# tootltips

In [8]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":0,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    71  100    71    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543


# Imports

In [552]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [636]:
def clean_desc_txt(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t",  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [653]:
def setup_es_client():
    es_client = Elasticsearch()

    index_name = "dpe_extract_text"
    index_configurations = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "max_result_window": 5000000,

            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "text_to_analyze": {
                    "type": "text",
                    "analyzer": "french"      
                },
                "category_index":{
                    'type':'keyword'
                }
            }
        }
    }

    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass
    es_client.indices.create(
        index=index_name,
        body=index_configurations
    )
    return es_client

# load tables

In [799]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths


if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='94':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td012_p = pd.read_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv', dtype=str)
            td014_p = pd.read_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv', dtype=str)
            td001_sys_ch= pd.read_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv', dtype=str)
            td001_sys_ecs= pd.read_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            td001_sys_ch.columns = [col.replace('_chauffage','_ch') for col in td001_sys_ch]
            print(dept_dir)
            break
            # ENVELOPPE PROCESSING


D:\data\dpe_full\depts\94
D:\data\dpe_full\depts\94


# ECS

In [655]:
mixte = ('mixte', 'combine', 'chauffage ecs','lie ','combine ',"idem")
elec =('electricite', 'electrique','elec.')

gen_ecs_normalized_lib_matching_dict_ft = {
    "ecs thermodynamique electrique(PAC ou ballon)": [
        ('pompe AND chaleur', 'pac', 'thermodynamique', 'air extrait', 'air exterieur', 'air ambiant')],
    
    "ballon a accumulation electrique": [('ballon', 'classique', 'accumulation'), elec],
    "ecs electrique indeterminee": [elec],
    "ecs instantanee electrique": ['instantanee', elec],
    

}

for type_chaudiere, type_chaudiere_keys in zip(['standard', 'basse temperature', 'condensation', 'non déterminee'],
                                               [('standard', 'classique'), 'basse temperature',
                                                ('condensation', 'condenseurs'), None]):
    for energie in ['fioul', 'gaz','autre(gpl/butane/propane)']:
        energie_keywords = energie
        if energie == 'autre(gpl/butane/propane)':
            energie_keywords = ('gpl', 'butane', 'propane')
        if type_chaudiere_keys is not None:
            gen_ecs_normalized_lib_matching_dict_ft[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords,
                                                                                            type_chaudiere_keys]
        else:
            gen_ecs_normalized_lib_matching_dict_ft[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords
                                                                                            ]
    gen_ecs_normalized_lib_matching_dict_ft[f'chaudiere energie indetermine {type_chaudiere}']=['chaudiere',type_chaudiere_keys] 


gen_ecs_normalized_lib_matching_dict_ft.update({    'chaudiere non determinee': ["chaudiere"],
        "ecs collective reseau chaleur": ["reseau de chaleur"],
    'production mixte gaz': ["gaz", mixte],
    'production mixte fioul': ["fioul", mixte],
    
    'production mixte indeterminee': [ mixte],
 'chauffe-eau gaz independant': [("individuelle AND ballon", "chauffe-eau", "accumulateur", "chauffe AND bain"), "gaz"],
    'chauffe-eau gpl independant': [("individuelle AND ballon", "chauffe-eau", "accumulateur", "chauffe AND bain"), "gpl"],

    'chauffe-eau fioul independant': [("individuelle AND ballon", "chauffe-eau", "accumulateur", "chauffe AND bain"),
                                      "fioul"],
                                                
    "chauffe-eau independant indetermine":[("individuelle AND ballon", "chauffe-eau", "accumulateur", "chauffe AND bain")]
})

In [656]:
from generate_dpe_annexes_scripts.utils import strip_accents

In [657]:
def gendata(index_name,id_data,data):
    for id_,vr in zip(id_data,data):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

In [658]:
char_list = ("individuelle AND ballon", "chauffe-eau", "accumulateur", "chauffe AND bain"), "gaz"

In [671]:
def search_from_search_dict(search_dict):
    
    s_list = list()
    for label,char_list in  search_dict.items():
        new_char_list=list()
        for char in char_list:
            if isinstance(char,str):
                new_char_list.append(char)
            if isinstance(char,tuple):
                new_char_list.append('('+') OR ('.join(char)+')')
        search_instruction = '('+') AND ('.join(new_char_list) +')'
        search_body = {
            "query": {
                "query_string": {
                    "query": search_instruction,
                    "default_field": "text_to_analyze"
                },

            },

        }
        print(label)
        print(search_instruction)
        #es_client.count(index=index_name, body=search_body)

        a_dict=es_client.search(index=index_name, body=search_body,size=500000)

        hits = a_dict['hits']['hits']

        s=pd.Series(index=[el['_source']['id'] for el in hits])
        s[:]=label
        s_list.append(s)
    s_all=pd.concat(s_list)

    return s_all

In [662]:
def search_and_affect(data,id_col,val_col,search_dict):
    es_client = setup_es_client()
    bulk(es_client, gendata(index_name,data[id_col],data[val_col]))
    s_all = search_from_search_dict(search_dict)

    df=s_all.to_frame('label')

    df.index.name = 'id'

    df = df.reset_index().drop_duplicates(keep='first',subset=['id'])

    m=data.merge(df,on='id',how='left')

    m.label=m.label.fillna('indetermine')
    return m

In [663]:

td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
td005_ecs.descriptif = td005_ecs.valeur_renseignee.fillna('indetermine')

td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.apply(lambda x:clean_desc_txt(x))

m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_normalized_lib_matching_dict_ft)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [664]:
gen_ecs_lib_ft=m.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'valeur_renseignee':lambda x: ' + '.join(sorted(list(set(x))))
    })
    

gen_ecs_lib_ft.columns =['gen_ecs_lib_ft','vr_source_ft']

In [666]:
td003_ecs = td003.loc[td003.tr007_type_descriptif_id == '10']
td003_ecs.descriptif = td003_ecs.descriptif.fillna('indetermine')
td003_ecs.descriptif = td003_ecs.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ecs.descriptif = td003_ecs.descriptif.apply(lambda x:clean_desc_txt(x))

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=gen_ecs_normalized_lib_matching_dict_ft)




gen_ecs_lib_desc=m.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'descriptif':lambda x: ' + '.join(sorted(list(set(x))))
    })
gen_ecs_lib_desc.columns =['gen_ecs_lib_desc','vr_source_desc']




C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [ ]:
m=gen_ecs_lib_desc.merge(gen_ecs_lib_ft,on='td001_dpe_id')
m=gen_ch_lib_ft.merge(grnv[['td001_dpe_id','gen_ch_lib_simp','type_installation_ch']],on='td001_dpe_id')
m=m.merge(gen[['td001_dpe_id','nom_methode_dpe_norm']],on='td001_dpe_id')
m=m.merge(type_installation_ch_ft.reset_index()[['td001_dpe_id','type_installation_ch_ft']])
m=m.merge(gen_ch_lib_desc,on='td001_dpe_id')
m=m.merge(type_installation_ch_desc.reset_index()[['td001_dpe_id','type_installation_ch_desc']])



In [669]:
ind_ft = gen_ecs_lib.gen_ecs_lib_ft=='indetermine'
ind_desc = gen_ecs_lib.gen_ecs_lib_desc=='indetermine'


In [670]:
gen_ecs_lib.loc[ind_ft&ind_desc].to_dict(orient='records')

[{'gen_ecs_lib_desc': 'indetermine',
  'vr_source_desc': 'aucune installation collective',
  'gen_ecs_lib_ft': 'indetermine',
  'vr_source_ft': 'aucune installation collective'},
 {'gen_ecs_lib_desc': 'indetermine',
  'vr_source_desc': 'production collective',
  'gen_ecs_lib_ft': 'indetermine',
  'vr_source_ft': 'production collective<br  '},
 {'gen_ecs_lib_desc': 'indetermine',
  'vr_source_desc': 'production collective',
  'gen_ecs_lib_ft': 'indetermine',
  'vr_source_ft': 'production collective<br  '},
 {'gen_ecs_lib_desc': 'indetermine',
  'vr_source_desc': 'production collective',
  'gen_ecs_lib_ft': 'indetermine',
  'vr_source_ft': 'production collective<br  '},
 {'gen_ecs_lib_desc': 'indetermine',
  'vr_source_desc': 'production collective',
  'gen_ecs_lib_ft': 'indetermine',
  'vr_source_ft': 'production collective<br  '},
 {'gen_ecs_lib_desc': 'indetermine',
  'vr_source_desc': 'production collective',
  'gen_ecs_lib_ft': 'indetermine',
  'vr_source_ft': 'production collective

# Chauffage

In [803]:


pac = ('pompe AND chaleur', 'pac', 'thermodynamique')
elec =('electricite', 'electrique','electriques','elec.','joule','joules')

installation_dict={'Chauffage Collectif':[('collective','collectif')],
                                 'Chauffage Individuel':[('individuelle','individuel')],
                                }

energie_dict = {
               'Gaz naturel':['gaz'],
               'Electricité non renouvelable':[elec],
                'Réseau de chaleurs':['reseau', 'chaleur'],
                'Fioul domestique':['fioul'],
    'Bois, biomasse':[('bois','biomasse')],
    'Gaz propane ou butane':[('propane','butane','gpl')],
    'Charbon':['charbon'],               
               }


energie_dict_lower = {
               'Gaz naturel':'gaz',
               'Electricité non renouvelable':'fioul',
                'Réseau de chaleurs':'reseau de chaleur',
                'Fioul domestique':'fioul',
    'Bois, biomasse':'bois',
    'Gaz propane ou butane':'autre(gpl/butane/propane)',
    'Charbon':'charbon',               
               }

type_chaudiere = dict(zip(['standard', 'basse temperature', 'condensation'],
                                               [('standard', 'classique'), 'basse temperature',
                                                ('condensation', 'condenseurs')]))

In [804]:

# GEN CH NORMALIZED DICT
gen_ch_normalized_lib_matching_dict_ft = {"pac air/air": [pac, 'air'],
                                        "pac eau/eau": [pac, 'eau'],

                                       "pac air/eau": [pac, 'air AND eau'],
                                       "pac geothermique": [pac, (
                                           'geothermique', 'geothermie')],
                                       'panneaux rayonnants electriques nfc': [('panneau','panneaux'),
                                                                               ('rayonnant','rayonnants'),
                                                                               'nfc'],
                                       'radiateurs electriques': [('radiateur','radiateurs'), elec],
                                       'plafonds/planchers rayonnants electriques nfc': [('plancher', 'plafond',
                                                                                          'planchers','plafonds'),
                                                                                         ('rayonnant','rayonnants')],
                                       "convecteurs electriques nfc": [('convecteur','convecteurs'),
                                                                       'nfc'],
                                       "poele ou insert bois": [('poele', 'insert'), ('bois', 'biomasse')],
                                       "poele ou insert fioul/gpl": [('poele', 'insert'), ('fioul', 'gpl')],
                                       "chaudiere bois": ['chaudiere',('bois','biomasse')],
                                       "autres emetteurs a effet joule": [elec],
                                       "reseau de chaleur": ['reseau', 'chaleur'],
                                       "convecteurs bi-jonction": ['bi', 'jonction',elec],
                                       }

for type_chaudiere, type_chaudiere_keys in zip(['standard', 'basse temperature', 'condensation', 'indetermine'],
                                               [('standard', 'classique'), 'basse temperature',
                                                ('condensation', 'condenseurs'), None]):
    for energie in ['fioul', 'gaz','autre(gpl/butane/propane)']:
        energie_keywords = energie
        if energie == 'autre(gpl/butane/propane)':
            energie_keywords = ('gpl', 'butane', 'propane')
        if type_chaudiere_keys is not None:
            gen_ch_normalized_lib_matching_dict_ft[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords,
                                                                                            type_chaudiere_keys]
        else:
            gen_ch_normalized_lib_matching_dict_ft[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords
                                                                                            ]
    gen_ch_normalized_lib_matching_dict_ft[f'chaudiere energie indetermine {type_chaudiere}']=['chaudiere',type_chaudiere_keys] 

gen_ch_normalized_lib_matching_dict_ft['chaudiere bois/biomasse'] = ['chaudiere',
                                                                  ('bois', 'biomasse')]
gen_ch_normalized_lib_matching_dict_ft['chaudiere electrique'] = ['chaudiere',
                                                               ('electricite', 'electrique')]






In [805]:

td005_ch = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '16']
td005_ch.descriptif = td005_ch.valeur_renseignee.fillna('indetermine')

td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.apply(lambda x:clean_desc_txt(x))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [806]:

m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ch_normalized_lib_matching_dict_ft)

gen_ch_lib_ft=m.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'valeur_renseignee':lambda x: ' + '.join(sorted(list(set(x))))
    })
    

gen_ch_lib_ft.columns =['gen_ch_lib_ft','vr_source_ft']

type_installation_ch=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ch_ft=type_installation_ch.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'valeur_renseignee':lambda x: ' + '.join(sorted(list(set(x))))
    })
type_installation_ch_ft.columns =['type_installation_ch_ft','vr_source_ft']  


energie_ch=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict)

energie_ch_ft=energie_ch.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'valeur_renseignee':lambda x: ' + '.join(sorted(list(set(x))))
    })
energie_ch_ft.columns =['energie_ch_ft','vr_source_ft']  




pac air/air
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND (air)
pac eau/eau
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND (eau)
pac air/eau
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND (air AND eau)
pac geothermique
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND ((geothermique) OR (geothermie))
panneaux rayonnants electriques nfc
((panneau) OR (panneaux)) AND ((rayonnant) OR (rayonnants)) AND (nfc)
radiateurs electriques
((radiateur) OR (radiateurs)) AND ((electricite) OR (electrique) OR (electriques) OR (elec.) OR (joule) OR (joules))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


plafonds/planchers rayonnants electriques nfc
((plancher) OR (plafond) OR (planchers) OR (plafonds)) AND ((rayonnant) OR (rayonnants))
convecteurs electriques nfc
((convecteur) OR (convecteurs)) AND (nfc)
poele ou insert bois
((poele) OR (insert)) AND ((bois) OR (biomasse))
poele ou insert fioul/gpl
((poele) OR (insert)) AND ((fioul) OR (gpl))
chaudiere bois
(chaudiere) AND ((bois) OR (biomasse))
autres emetteurs a effet joule
((electricite) OR (electrique) OR (electriques) OR (elec.) OR (joule) OR (joules))
reseau de chaleur
(reseau) AND (chaleur)
convecteurs bi-jonction
(bi) AND (jonction) AND ((electricite) OR (electrique) OR (electriques) OR (elec.) OR (joule) OR (joules))
chaudiere fioul standard
(chaudiere) AND (fioul) AND ((standard) OR (classique))
chaudiere gaz standard
(chaudiere) AND (gaz) AND ((standard) OR (classique))
chaudiere autre(gpl/butane/propane) standard
(chaudiere) AND ((gpl) OR (butane) OR (propane)) AND ((standard) OR (classique))
chaudiere energie indetermine 

In [807]:
td003_ch = td003.loc[td003.tr007_type_descriptif_id == '11']
td003_ch.descriptif = td003_ch.descriptif.fillna('indetermine')
td003_ch.descriptif = td003_ch.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ch.descriptif = td003_ch.descriptif.apply(lambda x:clean_desc_txt(x))

m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=gen_ch_normalized_lib_matching_dict_ft)


gen_ch_lib_desc=m.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'descriptif':lambda x: ' + '.join(sorted(list(set(x))))
    })
gen_ch_lib_desc.columns =['gen_ch_lib_desc','vr_source_desc']


type_installation_ch=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict={'Chauffage Collectif':[('collective','collectif')],
                                 'Chauffage Individuel':[('individuelle','individuel')],
                                })

type_installation_ch_desc=type_installation_ch.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'descriptif':lambda x: ' + '.join(sorted(list(set(x))))
    })
type_installation_ch_desc.columns =['type_installation_ch_desc','vr_source_desc']    

energie_ch=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict={'Chauffage Collectif':[('collective','collectif')],
                                 'Chauffage Individuel':[('individuelle','individuel')],
                                })

energie_ch_desc=energie_ch.groupby('td001_dpe_id').agg({

    'label':lambda x: ' + '.join(sorted(list(set(x)))),
    'descriptif':lambda x: ' + '.join(sorted(list(set(x))))
    })
energie_ch_desc.columns =['energie_ch_desc','vr_source_desc']    

pac air/air
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND (air)
pac eau/eau
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND (eau)
pac air/eau
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND (air AND eau)
pac geothermique
((pompe AND chaleur) OR (pac) OR (thermodynamique)) AND ((geothermique) OR (geothermie))
panneaux rayonnants electriques nfc
((panneau) OR (panneaux)) AND ((rayonnant) OR (rayonnants)) AND (nfc)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


radiateurs electriques
((radiateur) OR (radiateurs)) AND ((electricite) OR (electrique) OR (electriques) OR (elec.) OR (joule) OR (joules))
plafonds/planchers rayonnants electriques nfc
((plancher) OR (plafond) OR (planchers) OR (plafonds)) AND ((rayonnant) OR (rayonnants))
convecteurs electriques nfc
((convecteur) OR (convecteurs)) AND (nfc)
poele ou insert bois
((poele) OR (insert)) AND ((bois) OR (biomasse))
poele ou insert fioul/gpl
((poele) OR (insert)) AND ((fioul) OR (gpl))
chaudiere bois
(chaudiere) AND ((bois) OR (biomasse))
autres emetteurs a effet joule
((electricite) OR (electrique) OR (electriques) OR (elec.) OR (joule) OR (joules))
reseau de chaleur
(reseau) AND (chaleur)
convecteurs bi-jonction
(bi) AND (jonction) AND ((electricite) OR (electrique) OR (electriques) OR (elec.) OR (joule) OR (joules))
chaudiere fioul standard
(chaudiere) AND (fioul) AND ((standard) OR (classique))
chaudiere gaz standard
(chaudiere) AND (gaz) AND ((standard) OR (classique))
chaudiere autre(

In [816]:
m=gen_ch_lib_ft.reset_index()
m=m.merge(type_installation_ch_ft.reset_index()[['td001_dpe_id','type_installation_ch_ft']])
m=m.merge(energie_ch_ft.reset_index()[['td001_dpe_id','energie_ch_ft']])
m=m.merge(gen_ch_lib_desc,on='td001_dpe_id')
m=m.merge(type_installation_ch_desc.reset_index()[['td001_dpe_id','type_installation_ch_desc']])
m=m.merge(energie_ch_desc.reset_index()[['td001_dpe_id','energie_ch_desc']])
m=m.merge(td001_sys_ch[['td001_dpe_id','gen_ch_lib_infer_concat','mix_energetique_ch','type_installation_ch_concat']],on='td001_dpe_id')

m=m.merge(gen[['td001_dpe_id','nom_methode_dpe_norm']],on='td001_dpe_id')


In [818]:
m[['gen_ch_lib_ft','gen_ch_lib_desc','gen_ch_lib_infer_concat']]

,gen_ch_lib_ft,gen_ch_lib_desc,gen_ch_lib_infer_concat
0,convecteurs electriques nfc,convecteurs electriques nfc,convecteurs electriques nfc
1,convecteurs electriques nfc,convecteurs electriques nfc,autres emetteurs a effet joule
2,panneaux rayonnants electriques nfc,panneaux rayonnants electriques nfc,autres emetteurs a effet joule
3,chaudiere gaz standard,chaudiere gaz indetermine,chaudiere gaz standard
4,convecteurs electriques nfc,convecteurs electriques nfc,convecteurs electriques nfc
...,...,...,...
46092,chaudiere energie indetermine condensation,chaudiere energie indetermine condensation,chaudiere gaz indetermine
46093,chaudiere fioul standard,chaudiere energie indetermine standard,chaudiere fioul standard
46094,convecteurs electriques nfc,convecteurs electriques nfc,autres emetteurs a effet joule
46095,convecteurs electriques nfc,convecteurs electriques nfc,autres emetteurs a effet joule


In [810]:
m_ch = m

In [760]:
m.type_installation_ch_ft.value_counts()

indetermine                                                 49958
Chauffage Individuel                                        39845
Chauffage Collectif                                         33821
Chauffage Individuel + indetermine                            226
Chauffage Collectif + Chauffage Individuel                    198
Chauffage Collectif + indetermine                              24
Chauffage Collectif + Chauffage Individuel + indetermine        1
Name: type_installation_ch_ft, dtype: int64

In [762]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')

[{'td001_dpe_id': '1002110',
  'gen_ch_lib_ft': 'indetermine',
  'vr_source_ft': 'installation de chauffage  20\xa0m²)\xa0:\r - chauffage 1 : radiateurs gaz  energie : gaz)  combustion :  standard  fabrique e) avant 2006  pas de regulation sur generateur  sans equipement d intermittence',
  'gen_ch_lib_simp': nan,
  'type_installation_ch': nan,
  'nom_methode_dpe_norm': 'FACTURE',
  'type_installation_ch_ft': 'indetermine',
  'gen_ch_lib_desc': 'indetermine',
  'vr_source_desc': 'installation de chauffage \r - chauffage 1  radiateurs gaz  energie gaz naturel  reseau individuel ; annee de fabrication : avant 2006',
  'type_installation_ch_desc': 'Chauffage Individuel'},
 {'td001_dpe_id': '1004262',
  'gen_ch_lib_ft': 'indetermine',
  'vr_source_ft': 'poele installe apres 2001 avec label flamme verte  systeme individuel)<br  ',
  'gen_ch_lib_simp': nan,
  'type_installation_ch': nan,
  'nom_methode_dpe_norm': 'FACTURE',
  'type_installation_ch_ft': 'Chauffage Individuel',
  'gen_ch_lib_d

# fusion  